## 전체 음성 데이터 전처리

전체 아기울음 소리 음성 데이터(.wav)에 대해 전처리를 수행한다.

음성의 진폭과 주파수에 따른 특징이 잘 드러나도록 세그먼트 분할, 노이즈 제거, 등의 작업을 1~7 번 과정에서 수행하며

8~10 번 과정에서 음성 길이 통일, masking, 등의 과정을 통해 동일한 크기의 벡터 생성 및 데이터 증강을 수행한다.

본 문서는 수집된 전체 음성 데이터에 대해 수행되며 각 단계에 대한 필요성 언급 및 시각화를 하지 않는다. 이는 전체 데이터를 효과적으로 전처리하기 위함이며 각 단계의 필요성 및 시각적 자료는 trans_data_preview.ipynb 파일에 명시되어 있다.

<br>

전처리 순서는 아래와 같다.

1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다.

2. sample rate를 16000으로 통일한다.

3. 로그 멜 스펙트럼을 통해 음성의 파워를 측정한다.

4. 파워의 지역 최솟값을 기준으로 음성을 분할한다.

5. 분할된 음성의 앞뒤 화이트 노이즈(특정 임계값 이하의 에너지를 가지는 시점)을 제거한다.

6. 분할된 음성이 노이즈를 제거하여 패턴을 보다 잘 드러나게 한다.

7. 음성 정규화를 수행한다.

8. Padding과 Trimming을 통해 평균 세그멘테이션 길이를 가지도록 음성의 길이를 통일한다.

9. 음성의 유사도를 측정하여 하위 10%의 데이터는 제외한다.

10. 유사도 상위 10% 데이터에 대하여 SpecAugment를 적용하여 Training 데이터에 대해 마스킹 작업을 수행한다. 각 파일 당 마스킹 파일 2개를 생성한다.


In [1]:
# Load packages
import os
import sys
import wave
import librosa
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import soundfile as sf
import matplotlib.pyplot as plt
from uuid import uuid4

In [2]:
# Set Path
main_path = os.path.join(os.getcwd().rsplit(
    'baby-cry-classification')[0], 'baby-cry-classification')
data_path = os.path.join(main_path, 'data')
temp_data_path = os.path.join(main_path, 'temp_data')
csv_path = os.path.join(main_path, 'origin_data_info.csv')
origin_data_path = os.path.join(main_path, 'origin_data')

sys.path.append(main_path)

In [3]:
# 하이퍼 파라미터

search_in_sec = 3   # 파워값을 측정하는 시간 간격

state_list = ['sad', 'hug', 'diaper', 'hungry',
              'sleepy', 'awake', 'uncomfortable']

In [5]:
# 시각화 함수들을 정의한다.
def show_spectrum(y, sr):
    # STFT 계산
    D = librosa.amplitude_to_db(abs(librosa.stft(y)), ref=np.max)

    # 시간과 주파수 축을 위한 값들 계산
    times = np.linspace(0, len(y)/sr, num=D.shape[1], endpoint=False)
    freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)

    # 스펙트럼 시각화
    plt.figure(figsize=(8, 4))
    plt.imshow(D, aspect='auto', origin='lower', extent=[
               times.min(), times.max(), freqs.min(), freqs.max()])
    plt.colorbar(format='%+2.0f dB')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Spectrogram')
    plt.yscale('log')
    plt.tight_layout()
    plt.show()


def show_mel_power(power, dot_list=[], dot_color='red', dot_label=''):
    # 파워 시각화
    plt.figure(figsize=(12, 5))
    plt.plot(power)
    plt.title("Log Mel Spectrum Power")
    plt.xlabel("Time Frame")
    plt.ylabel("Power")

    if (len(dot_list) > 0):
        plt.scatter(dot_list, power[dot_list],
                    color=dot_color, label=dot_label)
        plt.legend()

    plt.grid(True)
    plt.tight_layout()
    plt.show()


def show_masked_mel(db_masked_mel):
    plt.figure(figsize=(10, 4))
    plt.imshow(db_masked_mel, origin='lower', aspect='auto',
               extent=[0, db_masked_mel.shape[1], 0, db_masked_mel.shape[0]])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Realistically Masked Mel spectrogram (using matplotlib)')
    plt.xlabel('Time frames')
    plt.ylabel('Mel frequency bins')
    plt.tight_layout()
    plt.show()

In [ ]:
# 1,2 번 과정을 수행한다.
from trans_data import create_state_folder, get_state_file_list

create_state_folder(origin_data_path, csv_path, temp_data_path, etc=False)
file_list = get_state_file_list(temp_data_path, state_list, '.wav')

file_list = get_state_file_list(temp_data_path, state_list, '.wav')

In [7]:
# 3번 과정의 함수를 정의한다.
from scipy.signal import stft


def read_wav(file_path):
    with wave.open(file_path, 'r') as wav_file:
        n_channels, sampwidth, framerate, n_frames, comptype, compname = wav_file.getparams()
        audio_data = wav_file.readframes(n_frames)
        audio_data = np.frombuffer(audio_data, dtype=np.int16)

    # Stereo 파일의 경우 mono로 변환
    if n_channels == 2:
        audio_data = (audio_data[::2] + audio_data[1::2]) / 2

    return [audio_data, {'n_channels': n_channels,
                         'sampwidth': sampwidth,
                         'framerate': framerate,
                         'n_frames': n_frames,
                         'comptype': comptype,
                         'compname': compname}]


def mel_filter_bank(num_filters, fft_size, sample_rate):
    """
    멜 필터뱅크 생성
    """
    # 멜 스케일과 헤르츠 스케일 간의 변환 함수
    def hz_to_mel(hz): return 2595 * np.log10(1 + hz / 700)
    def mel_to_hz(mel): return 700 * (10**(mel / 2595) - 1)

    # 멜 스케일로 끝점 설정
    mel_end = hz_to_mel(sample_rate / 2)
    mel_points = np.linspace(hz_to_mel(0), mel_end, num_filters + 2)
    hz_points = mel_to_hz(mel_points)

    # FFT 주파수 인덱스로 변환
    bin_points = np.floor((fft_size + 1) * hz_points / sample_rate).astype(int)

    # 필터뱅크 생성
    filters = np.zeros((num_filters, fft_size // 2 + 1))
    for i in range(1, num_filters + 1):
        filters[i - 1, bin_points[i - 1]:bin_points[i]] = \
            (np.arange(bin_points[i - 1], bin_points[i]) -
             bin_points[i - 1]) / (bin_points[i] - bin_points[i - 1])
        filters[i - 1, bin_points[i]:bin_points[i + 1]] = 1 - \
            (np.arange(bin_points[i], bin_points[i + 1]) -
             bin_points[i]) / (bin_points[i + 1] - bin_points[i])
    return filters


def get_log_mel_spectogram(audio_data, framerate, num_filters=40, nperseg=2048, noverlap=1024, nfft=2048):
    # STFT 계산
    _, _, Zxx = stft(audio_data, fs=framerate, nperseg=nperseg,
                     noverlap=noverlap, nfft=nfft)
    magnitude = np.abs(Zxx)

    # 로그 멜 스펙트럼 추출
    num_filters = num_filters
    filters = mel_filter_bank(num_filters, 2048, framerate)
    mel_spectrum = np.dot(filters, magnitude)
    log_mel_spectrum = np.log(mel_spectrum + 1e-9)  # log 0을 피하기 위한 작은 값 추가
    return log_mel_spectrum


def get_mel_power(log_mel_spectrum):
    # 로그 멜 스펙트럼의 파워 계산
    return np.sum(log_mel_spectrum**2, axis=0)

In [8]:
# 4번 과정의 함수를 정의한다.
def get_interval_min_sec(power, sec: int = 1):
    interval_sec = int(10 * sec)
    min_sec_list = []
    for i in range(0, len(power) + 1, interval_sec):
        argsort = np.argsort(power[i:i+interval_sec])
        if len(argsort) < 1:
            break
        min_sec = i + argsort[0]
        min_sec_list.append(min_sec)
    return min_sec_list


def split_audio_on_indices(audio_data, power, indices):
    """
    주어진 인덱스를 기준으로 오디오 데이터를 여러 부분으로 나누는 함수.

    Parameters:
    - audio_data: 오디오 데이터 배열
    - indices: 분할할 시점의 리스트

    Returns:
    - audio_segments: 나눈 오디오 데이터의 리스트
    """
    # STFT를 통해 구한 시점을 오디오 샘플의 시점으로 변환
    nperseg = 2048
    noverlap = 1024
    samples_per_frame = (len(audio_data) - nperseg) / (len(power) - 1)
    sample_indices = [int(index * samples_per_frame) for index in indices]

    audio_segments = []
    prev_index = 0
    for index in sample_indices:
        segment = audio_data[prev_index:index]
        if len(segment) != 0:
            audio_segments.append(segment)
        prev_index = index
    audio_segments.append(audio_data[prev_index:])  # 마지막 세그먼트 추가

    return audio_segments


def save_audio_segments_as_wav(audio_segments, output_dir, prefix, sampwidth, framerate):
    """
    주어진 오디오 세그먼트들을 WAV 파일로 저장하는 함수.

    Parameters:
    - audio_segments: 나눈 오디오 데이터의 리스트
    - output_dir: 출력 디렉토리 경로
    - prefix: 저장할 파일의 접두사

    Returns:
    - filepaths: 저장된 파일의 경로 리스트
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    filepaths = []
    for i, segment in enumerate(audio_segments):
        filename = f"{prefix}_{uuid4()}.wav"
        filepath = os.path.join(output_dir, filename)
        with wave.open(filepath, 'wb') as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(sampwidth)
            wav_file.setframerate(framerate)
            wav_file.writeframes(segment.tobytes())
        filepaths.append(filepath)

    return filepaths

In [9]:
# 5,6,7번 과정의 함수를 정의하거나 불러온다.

from trans_data import trim_audio, reduced_base_noise
from scipy.io import wavfile


def normalize_and_save_audio(input_path, output_path=None, inplace=True):
    """
    Load an audio file from the given input path, normalize it, and save the normalized audio to the given output path.

    Parameters:
    - input_path: Path to the input audio file.
    - output_path: Path to save the normalized audio file.
    - inplace: 이미 존재하는 파일에 덮어쓴다.
    """
    if inplace == False and output_path == None:
        raise ValueError(f'output path must be defined if inplace is False.')

    # Load the audio file
    y, sr = librosa.load(input_path, sr=None)

    # Normalize the audio data
    y_normalized = y / np.max(np.abs(y))

    # Convert float to int16 (since wavfile.write requires int values)
    y_normalized_int16 = (y_normalized * 32767).astype(np.int16)

    if inplace:
        os.remove(input_path)
        output_path = input_path

    # Save the normalized audio data to the specified output path
    wavfile.write(output_path, sr, y_normalized_int16)

In [10]:
# 3 ~ 7 번 과정을 수행한다.

for i in tqdm(range(len(file_list))):
    file_state = file_list[i].rsplit('/', 2)[1]

    # 3. 로그 멜 스펙트럼을 통해 음성의 파워를 측정한다.

    # 파일을 읽어온다.
    audio_data, audio_status = read_wav(file_list[i])

    # 로그 멜 스펙트럼을 추출한 뒤
    log_mel_spectogram = get_log_mel_spectogram(
        audio_data, audio_status['framerate'])

    # 파워를 측정한다.
    power = get_mel_power(log_mel_spectogram)

    # 4. 파워의 지역 최솟값을 기준으로 음성을 분할한다.

    # 파워의 지역 최솟값의 시점들을 구한다.
    min_sec_list = get_interval_min_sec(power, sec=search_in_sec)

    # 지역 최솟값을 기준으로 오디오(numpy)값을 분할한다.
    audio_segments_sec = split_audio_on_indices(
        audio_data, power, min_sec_list)

    # 분할된 값을 data_path에 저장한다.
    saved_filepaths = save_audio_segments_as_wav(
        audio_segments=audio_segments_sec,
        output_dir=data_path,
        prefix=file_state,
        sampwidth=audio_status['sampwidth'],
        framerate=audio_status['framerate'],
    )

    # # 분할된 음성 중 search_in_sec의 절반보다 긴 음성들에 대해 5,6 과정을 진행한다.
    total_error_files = []
    for file_path in saved_filepaths:
        # WAV 파일 로드
        y, sr = librosa.load(file_path, sr=None)

        # 파일의 길이(초) 출력
        origin_duration = librosa.get_duration(y=y, sr=sr)
        if origin_duration > 1:
            # 5. 분할된 음성의 앞뒤 화이트 노이즈(특정 임계값 이하의 에너지를 가지는 시점)을 제거한다.
            error_files = trim_audio(file_path, inplace=True, frame_size=1000)
            if (len(error_files) > 0):
                total_error_files += error_files

            # 6. 분할된 음성이 노이즈를 제거하여 패턴을 보다 잘 드러나게 한다.
            reduced_base_noise(file_path, inplace=True)

            # 7. 음성 정규화를 수행한다.
            normalize_and_save_audio(file_path, inplace=True)
        else:
            os.remove(file_path)

  0%|          | 0/3238 [00:00<?, ?it/s]/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)
  2%|▏         | 70/3238 [00:08<05:26,  9.71it/s]/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/noisereduce/noisereduce.py:306: RuntimeWarning: invalid value encountered in true_divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
/var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/ipykernel_97901/1225871274.py:22: RuntimeWarning: invalid value encountered in true_divide
  y_norm

### 음성 길이 규정

위 과정을 통해 음성의 특징을 가지는 각각의 전처리된 세그먼트가 wav 파일 형태로 저장되었다.

모델의 입력 벡터로 구성하기 위해서는 모든 음성의 길이가 동일해야 함으로 모든 세그먼트의 평균 길이를 구한 뒤 Padding과 Trimming 과정을 통해 평균 길이로 음성 길이를 통일하고자 한다.


#### 평균 세그먼트 길이 측정

아래 과정을 통해 평균 세그먼트의 (음성)길이는 1.86 임을 알았다. 이에 음성을 2초로 통일하고자 한다.


In [11]:
file_list = [os.path.join(data_path, file) for file in os.listdir(data_path)]
duration_list = []
for file_path in file_list:
    y, sr = librosa.load(file_path, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < 1:
        os.remove(file_path)
    else:
        duration_list.append(duration)
sum = np.array(duration_list).sum()
avg = sum / len(duration_list)
avg_duration = round(avg)
print(f'Average: {avg}')
print(f'사용할 음성의 길이: {avg_duration}')

Average: 1.8620317905047654
사용할 음성의 길이: 2


In [ ]:
# state(hungry, sad, ...etc) 이름을 가지는 폴더로 해당하는 음성 파일을 이동시킨 뒤 음성 파일의 이름을 state_숫자 형태로 변환하여
# 사용자가 파일명을 통한 state 확인을 돕는다.
from utils.os import move_file


def move_files_by_state(data_path, state_list):
    file_list = [os.path.join(data_path, file)
                 for file in os.listdir(data_path)]
    new_file_list = []

    for state in state_list:
        os.mkdir(os.path.join(data_path, state))

    for file in file_list:
        file_name = file.rsplit('/', 1)[1]
        file_state = file_name.split('_')[0]
        # new_file_path = f'{data_path}/{file_state}/{file_name}'
        new_file_path = os.path.join(data_path, file_state, file_name)
        move_file(file, new_file_path)
        new_file_list.append(new_file_path)
    return new_file_list


new_file_list = move_files_by_state(data_path, state_list)

In [13]:
# 8번 과정을 수행한다.

def pad_audio_to_length(input_path, target_duration, output_path=None, inplace=False):
    """
    wav 파일을 target_duration 길이로 맞춘다. 
    음성의 중심을 기준으로 target_duration보다 짧을 경우 앞뒤에 Padding을 추가하며 
    음성의 중심을 기준으로 target_duration보다 길 경우 앞뒤를 자른다.
    """

    if inplace == False and output_path == None:
        raise ValueError(f'output path must be defined if inplace is False.')

    # WAV 파일 읽기
    y, sr = librosa.load(input_path, sr=None)

    # 현재 오디오 파일의 길이 확인
    current_duration = len(y) / sr

    # target_duration과 비교하여 패딩 또는 trimming 필요 여부 확인
    total_samples = int(target_duration * sr)

    if len(y) < total_samples:
        # 패딩 필요
        padding_samples = total_samples - len(y)
        left_padding = padding_samples // 2
        right_padding = padding_samples - left_padding
        processed_audio = np.pad(
            y, (left_padding, right_padding), mode='constant')
    else:
        # trimming 필요
        excess_samples = len(y) - total_samples
        left_trim = excess_samples // 2
        right_trim = excess_samples - left_trim
        processed_audio = y[left_trim:-right_trim]

    if inplace:
        os.remove(input_path)
        output_path = input_path

    # 변환된 데이터를 WAV 파일로 저장
    sf.write(output_path, processed_audio, sr)

    return output_path


for file_path in new_file_list:
    pad_audio_to_length(file_path, avg_duration, inplace=True)

In [22]:
# 9번 과정을 수행한다 : 음성의 유사도를 측정하여 하위 10%의 데이터는 제외한다.

min_state = ''
min_count = 100000    # 초기값: 불가능한 아주 큰 수
for state in os.listdir(data_path):
    count = len(os.listdir(os.path.join(data_path, state)))
    print(f'State {state} with file counts: {count}')
    if count < min_count:
        min_count = count
        min_state = state

print(f'\n가장 적게 존재하는 State는 {state} 이며 개수는 {min_count} 이다.')

State sad with file counts: 3389
State hug with file counts: 1972
State diaper with file counts: 1349
State hungry with file counts: 2751
State sleepy with file counts: 1645
State awake with file counts: 1593
State uncomfortable with file counts: 1466

가장 적게 존재하는 State는 uncomfortable 이며 개수는 1349 이다.


In [ ]:
# 9번 과정을 수행한다 : 유사도를 측정한 뒤 전체 파일의 개수를 맞춘다.
from trans_data import get_similarities
from utils.os import remove_file


def delete_dissimilar_wavs(data_path: str, n_limit: int, save_dir=None):
    state_list = ['sad', 'hug', 'diaper', 'hungry',
                  'sleepy', 'awake', 'uncomfortable']
    for state in state_list:
        state_path = os.path.join(data_path, state)
        state_file_list = [os.path.join(state_path, file)
                           for file in os.listdir(state_path)]

        # get similarities
        sim_file_list = get_similarities(state_file_list)
        if (save_dir != None):
            np.savetxt(f'{os.path.join(save_dir, state)}_similarity.txt',
                       sim_file_list, delimiter=',', fmt='%s')

        # 상위 90%를 제외하고 모두 제거한다.
        del_file_list = sim_file_list[n_limit:]
        for file in del_file_list:
            remove_file(file)


save_dir = os.path.join(main_path, 'save_similarity')
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

delete_dissimilar_wavs(data_path, min_count, save_dir)

In [59]:
# 9번 과정을 수행한다 : 음성의 유사도를 측정하여 하위 10%의 데이터는 제외한다.
# 그리고 유사도 순서대로 파일에 순번을 부과한다.
left_file_len = int(min_count * 0.9) + 1

save_dir = os.path.join(main_path, 'save_similarity')
for txt_file in os.listdir(save_dir):

    with open(os.path.join(save_dir, txt_file), 'r') as f:
        file_list = f.read().splitlines()

    state = txt_file.split('_')[0]

    del_file_list = file_list[left_file_len:]
    for file in del_file_list:
        if os.path.exists(file):
            os.remove(file)

    use_file_list = file_list[:left_file_len]

    # rename
    renamed_file_list = []
    for i in range(len(use_file_list)):
        path = use_file_list[i].rsplit('/', 1)[0]
        ex = use_file_list[i].rsplit('.', 1)[1]
        renamed_file = f'{path}/{state}_{i+1}.{ex}'
        renamed_file_list.append(renamed_file)
        os.rename(use_file_list[i], renamed_file)

    print(f'Process {txt_file} Done.')

Process uncomfortable_similarity.txt Done.
Process sad_similarity.txt Done.
Process hug_similarity.txt Done.
Process hungry_similarity.txt Done.
Process awake_similarity.txt Done.
Process diaper_similarity.txt Done.
Process sleepy_similarity.txt Done.


In [ ]:
# 처리가 완료된 유사도 파일들은 삭제한다.
from utils.os import remove_path_with_files

remove_path_with_files(save_dir)

In [7]:
# 10번 과정을 수행한다.
# 유사도 상위 10% 데이터에 대하여 SpecAugment를 적용하여 Training 데이터에 대해 마스킹 작업을 수행한다.

def mask_melspectrogram(file_path,
                        freq_mask_num=2,
                        time_mask_num=2,
                        freq_masking_max_percentage=0.15,
                        time_masking_max_percentage=0.3):

    # 1. Load the audio file
    y, sr = librosa.load(file_path, sr=None)

    # 2. Extract mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    num_freq_bins = mel_spectrogram.shape[0]
    num_time_bins = mel_spectrogram.shape[1]

    # 3. Apply masking
    # Frequency masking
    for _ in range(freq_mask_num):
        freq_start = np.random.randint(0, num_freq_bins)
        freq_length = np.random.randint(
            0, int(num_freq_bins * freq_masking_max_percentage))
        freq_end = min(mel_spectrogram.shape[0], freq_start + freq_length)
        mel_spectrogram[freq_start:freq_end, :] = 0

    # Time masking
    for _ in range(time_mask_num):
        time_start = np.random.randint(0, num_time_bins)
        time_length = np.random.randint(
            0, int(num_time_bins * time_masking_max_percentage))
        time_end = min(mel_spectrogram.shape[1], time_start + time_length)
        mel_spectrogram[:, time_start:time_end] = 0

    return mel_spectrogram


def show_masked_mel(db_masked_mel):
    plt.figure(figsize=(10, 4))
    plt.imshow(db_masked_mel, origin='lower', aspect='auto',
               extent=[0, db_masked_mel.shape[1], 0, db_masked_mel.shape[0]])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Realistically Masked Mel spectrogram (using matplotlib)')
    plt.xlabel('Time frames')
    plt.ylabel('Mel frequency bins')
    plt.tight_layout()
    plt.show()


def save_masked_spectrogram_to_wav(mel_spectrogram, sr, save_path):
    """ Convert masked mel spectrogram back to waveform and save as wav """
    y_inv = librosa.feature.inverse.mel_to_audio(mel_spectrogram, sr=sr)
    sf.write(save_path, y_inv, sr)


def create_mask(wav_file, output_dir=None, n_create=2):
    sr = 16000
    wav_output_dir, wav_file_name = wav_file.rsplit('/', 1)
    base_filename = wav_file_name.split('.')[0]

    if output_dir == None:
        output_dir = wav_output_dir

    for i in range(1, n_create + 1, 1):
        odd = i % 2 == 0
        masked_mel = mask_melspectrogram(
            wav_file,
            freq_mask_num=1,
            time_mask_num=1,
            freq_masking_max_percentage=0.12 if odd else 0.07,
            time_masking_max_percentage=0.07 if odd else 0.12
        )
        save_masked_spectrogram_to_wav(masked_mel, sr, os.path.join(
            output_dir, base_filename + f"_mask{i}.wav"))
        # show_masked_mel(librosa.power_to_db(masked_mel, ref=np.max))
        # print(f'Save: {os.path.join(output_dir, base_filename + f"_mask{i}.wav")}')


top_10 = int(len(os.listdir(os.path.join(data_path, state_list[0]))) * 0.1)

for state in os.listdir(data_path):
    if state == '.DS_Store':
        continue

    state_data_path = os.path.join(data_path, state)
    file_list = os.listdir(state_data_path)
    for i in tqdm(range(1, top_10 + 1, 1)):
        create_mask(
            wav_file=os.path.join(state_data_path, f'{state}_{i}.wav'),
            output_dir=state_data_path,
            n_create=1
        )
    print(f'State {state} masking done.')

100%|██████████| 121/121 [00:30<00:00,  3.94it/s]


State sad masking done.


100%|██████████| 121/121 [00:42<00:00,  2.87it/s]


State hug masking done.


100%|██████████| 121/121 [00:37<00:00,  3.26it/s]


...

State sleepy masking done.


100%|██████████| 121/121 [00:40<00:00,  2.99it/s]


State awake masking done.


100%|██████████| 121/121 [00:47<00:00,  2.55it/s]

State uncomfortable masking done.
